<a href="https://colab.research.google.com/github/Alexey1998-ml/parse/blob/master/parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 7.1MB 7.5MB/s 


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
import requests
from bs4 import BeautifulSoup
import urllib
import pickle
import os

import google.oauth2.credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import googleapiclient.discovery

from nltk import word_tokenize
from nltk.corpus import stopwords

import pymorphy2

import pandas as pd
import numpy as np


from gensim.models import FastText


from sklearn.manifold import TSNE

In [0]:
CLIENT_SECRETS_FILE = "client_secret.json"
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'

In [0]:
def get_authenticated_service():
    credentials = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)
    #  Check if the credentials are invalid or do not exist
    if not credentials or not credentials.valid:
        # Check if the credentials have expired
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_console()
 
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)
 
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)
 

In [6]:
service = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=499464782258-sk1tjt846grh4bartqo42538n81bkb8o.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=OhxNQrwjGMQnizxPOcyBZVRJ9M6AQj&prompt=consent&access_type=offline
Enter the authorization code: 4/zAE2VbFjWLLnxfxhayfb3hfPL5JySWe1R4PxBxay3QMDiUaqcsTHmy8


In [0]:
def get_video_comments(service, **kwargs):
    summary = []
    results = service.commentThreads().list(**kwargs).execute()
    
    while results:
        for item in results['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            num_like = item['snippet']['topLevelComment']['snippet']['likeCount']
            num_reply = item['snippet']['totalReplyCount']
            summary.append({
                'comment' : comment,
                'num_like' : num_like,
                'num_reply' : num_reply
            })
 
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.commentThreads().list(**kwargs).execute()
        else:
            break
 
    return pd.DataFrame(summary)

In [0]:
def check_word(word):
    if word.isalpha() and word not in stop_words:
        return morph.parse(word)[0].normal_form

    
def tokenize_sentence(sentence):
    words = word_tokenize(sentence)
    norm_words = []
    for word in words:
        cur_word = check_word(word)
        if cur_word is not None:
            norm_words.append(cur_word)
    if len(norm_words) > 0:
      return norm_words
    return None


def add_tokenize_columns(df, tokenized_col='tokenize_text', text_for_tokenize='comment', language='russian'):
    global stop_words, morph
    stop_words = set(stopwords.words(language))
    morph = pymorphy2.MorphAnalyzer()
    df[tokenized_col] = df[text_for_tokenize].apply(tokenize_sentence)
    return df.dropna()

In [0]:
summary = get_video_comments(service, part='snippet', videoId='kOvdx1sz--U', textFormat='plainText')

In [0]:
summary = add_tokenize_columns(summary)

In [0]:
def make_model(df, token_tolumn = 'tokenize_text', size = 128, window = 3, min_count = 1, epochs = 10):
  model = FastText(size=size, window=window, min_count=min_count)
  model.build_vocab(df[token_tolumn])
  model.train(sentences=df[token_tolumn], total_examples=len(df[token_tolumn]), epochs=epochs) 
  return model

In [0]:
model = make_model(summary)

In [0]:
def get_mean_embedding(string):
  embedding = np.array(list(map(model.__getitem__, string))).mean(axis = 0)
  if string:
    return list(embedding)

In [255]:
summary['mean_embedding'] = summary['tokenize_text'].apply(get_mean_embedding)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
tsne = TSNE(n_components = 2)
mean_embedding_transform = tsne.fit_transform(np.asarray(summary['mean_embedding'].to_list()))

In [0]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 color_target = 'red',
                 width=600, height=400, show=True, **kwargs):
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })
    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)

    # fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [263]:
draw_vectors(mean_embedding_transform[:, 0], mean_embedding_transform[:, 1])

Figure(id='1484', ...)